In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as seabornInstance
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.model_selection import LeaveOneOut
from sklearn import preprocessing
from sklearn import svm
from sklearn import metrics
%matplotlib inline

In [ ]:
data = pd.read_csv(r"C:\Users\sshin\Desktop\SYSHIN0228\RERC_STARTS_Sensor_Analysis\Machine_Learning_6MWT\MWT6_AoM_Feature_Matrix360.csv", sep=',')

# data.head()
# data.shape
data.describe()
# print(data)

In [ ]:
X = data[['AoM_Pel_tilt', 'AoM_Pel_ro', 'AoM_Pel_oblq', 'AoM_Pel_Norm',
        'AoM_Ankle_US_x', 'AoM_Ankle_US_y', 'AoM_Ankle_US_z', 'AoM_Ankle_US_Norm',
        'AoM_Ankle_AS_x', 'AoM_Ankle_AS_y', 'AoM_Ankle_AS_z', 'AoM_Ankle_US_Norm']].values

y = data['MWT6'].values.reshape(-1,1)

min_max_scaler = preprocessing.MinMaxScaler()
X_scale = min_max_scaler.fit_transform(X)
y_scale = min_max_scaler.fit_transform(y)

# X_scale = preprocessing.scale(X)
# y_scale = preprocessing.scale(y)

print(X_scale)
print(y_scale)

# k-Fold Cross Validation

In [ ]:
kf = KFold(n_splits=10, random_state=12, shuffle=False)
kf.get_n_splits(X)
print(kf)

In [ ]:
i = 1
RMSE = np.zeros((10))

for train_index, test_index in kf.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_scale[train_index], X_scale[test_index]
    y_train, y_test = y_scale[train_index], y_scale[test_index]
    
    regressor = svm.SVR(kernel = 'linear', C = 1, gamma='auto')
    regressor.fit(X_train,y_train)
    
    y_pred = regressor.predict(X_test)
    
    y_pred_ = y_pred*(max(y)-min(y))+min(y)
    y_test_ = y_test*(max(y)-min(y))+min(y)
    
    
    df = pd.DataFrame({'Actual': y_test_.flatten(), 'Predicted': y_pred_.flatten()})
    print(df)
    
    df1 = df.head(25)
    df1.plot(kind='bar', figsize=(16,10))
    plt.grid(which='major', linestyle='-', linewidth='0.5', color='red')
    # plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    plt.show()
    
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_, y_pred_))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test_, y_pred_))
    print('RMS:', np.sqrt(metrics.mean_squared_error(y_test_, y_pred_)))
    
    RMSE[i-1] = np.sqrt(metrics.mean_squared_error(y_test_, y_pred_))
    
    i = i+1
    print("Iteration: ", i)
    

print("RMSE:", RMSE)
RMSE_mean = np.mean(RMSE)
RMSE_std = np.std(RMSE)

print("Average RMSE:",RMSE_mean, "+-", RMSE_std)    


# Leave-one-out Cross Validation

In [ ]:
loo = LeaveOneOut()
loo.get_n_splits(X)
i = 1
RMSE = np.zeros((141))
for train_index, test_index in loo.split(X):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X_scale[train_index], X_scale[test_index]
    y_train, y_test = y_scale[train_index], y_scale[test_index]
    
    regressor = svm.SVR(kernel = 'linear', C = 1, gamma='auto')
    regressor.fit(X_train,y_train)
    
    y_pred = regressor.predict(X_test)
    
    y_pred_ = y_pred*(max(y)-min(y))+min(y)
    y_test_ = y_test*(max(y)-min(y))+min(y)
    
    
    df = pd.DataFrame({'Actual': y_test_.flatten(), 'Predicted': y_pred_.flatten()})
    df
    
#     df1 = df.head(25)
#     df1.plot(kind='bar', figsize=(16,10))
#     plt.grid(which='major', linestyle='-', linewidth='0.5', color='red')
#     # plt.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
#     plt.show()
    
    print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_, y_pred_))
    print('Mean Squared Error:', metrics.mean_squared_error(y_test_, y_pred_))
    print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test_, y_pred_)))
    
    RMSE[i-1] = np.sqrt(metrics.mean_squared_error(y_test_, y_pred_))
    
    i = i+1
    print("Iteration: ", i)
    

print("RMSE:", RMSE)
RMSE_mean = np.mean(RMSE)
RMSE_std = np.std(RMSE)

print("Average RMSE:",RMSE_mean, "+-", RMSE_std)
    